# Prepare data for PFD SPM analysis

This does the following:
* Loads CPSP's master historical SPM file, limiting to relevant fields (see [documentation](https://static1.squarespace.com/static/5743308460b5e922a25a6dc7/t/5c8179014785d342b6e63abe/1551988993650/SPM+public+use+data+documentation_02142019.pdf)).
* Merges with IPUMS ASEC data to get state.
* Outputs data with necessary fields.

## Setup

In [1]:
import pandas as pd
import numpy as np

## Load data

In [2]:
spm_raw = pd.read_csv('/home/mghenis/datarepo/pub_spm_master.csv.gz',
                      usecols=['year', 'serial', 'lineno', 'pernum',
                               'sex', 'age', 'a_sex', 'a_age', 
                               'SPMu_Poor_Metadj_anch_cen', 'marsupwt'])

/home/mghenis/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,41) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ipums_raw = pd.read_csv('asec_hh_state.csv.gz')

## Preprocess

### SPM

Recode sex and age. Use `female` for clarity.

In [4]:
spm = spm_raw.copy(deep=True)

In [5]:
spm['female'] = (spm.sex == 'Female') | (spm.a_sex == 'Female')
spm.age = np.where(spm.age.isnull(), spm.a_age, spm.age)
spm.drop(['a_age', 'a_sex', 'sex'], axis=1, inplace=True)

Weight is multiplied by 100 prior to 1991.

In [6]:
spm.marsupwt = np.where(spm.year < 1991, spm.marsupwt / 100, spm.marsupwt)

### IPUMS

In [7]:
ipums = ipums_raw.copy(deep=True)

In [8]:
ipums.columns = ipums.columns.map(str.lower)

In [9]:
ipums['female'] = ipums.sex == 2
ipums.drop(['sex'], axis=1, inplace=True)

In [10]:
# Recode year to be calendar rather than survey year.
ipums.year -= 1

## Merge

In [11]:
def merge_ipums_cpsp(ipums, cpsp):
    """ Merges IPUMS ASEC with CPSP historical SPM poverty file, per CPSP
        documentation.
    
    Args:
        ipums: Raw IPUMS ASEC. Must include year, serial, lineno, female, age, 
            and pernum.
        cpsp: Raw CPSP historical SPM poverty file, with female added from
            sex and a_sex, and age set to a_age when age is null.
        
    Returns:
        DataFrame with all relevant fields from ipums and cpsp.
        
    Note: This procedure is required even for household-level fields, as
        CPSID (a household identifier) was introduced in the 1989 survey year.
    """
    LINK_VARS_1975_1977 = ['year', 'serial', 'pernum', 'female', 'age']
    LINK_VARS_OTHER_YEARS = ['year', 'serial', 'lineno', 'female', 'age']
    # Create intermediate datasets to merge, dropping unused merge columns.
    ipums_1975_1977 = ipums[ipums.year.isin([1975, 1976, 1977])].drop(
        ['lineno'], axis=1)
    cpsp_1975_1977 = cpsp[cpsp.year.isin([1975, 1976, 1977])].drop(
        ['lineno'], axis=1)
    ipums_other_years = ipums[~ipums.year.isin([1975, 1976, 1977])].drop(
        ['pernum'], axis=1)
    cpsp_other_years = cpsp[~cpsp.year.isin([1975, 1976, 1977])].drop(
        ['pernum'], axis=1)
    # Merge.
    res_1975_1977 = ipums_1975_1977.merge(cpsp_1975_1977,
                                          on=LINK_VARS_1975_1977)
    res_other_years = ipums_other_years.merge(cpsp_other_years,
                                              on=LINK_VARS_OTHER_YEARS)
    # Return the concatenation of the two files after re-sorting on year.
    return pd.concat([res_1975_1977, res_other_years]).sort_values('year')

In [12]:
merged = merge_ipums_cpsp(ipums, spm)

## Clean up and export

Rename core columns.

In [13]:
merged.rename({'SPMu_Poor_Metadj_anch_cen': 'poor', 'marsupwt': 'w'}, axis=1,
              inplace=True)

Export without all the merging columns.

In [14]:
merged[['year', 'statefip', 'age', 'female', 'poor', 'w']].to_csv(
    'spm_state.csv.gz', index=False)